In [53]:
# Install all the necessary python libraries if anaconda is not able to find them.


In [2]:
# Import all the necessary python libraries
import re
import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Enable pandas integration
import numpy as np
# from fuzzywuzzy import fuzz, process
from thefuzz import fuzz


In [55]:
# Initialize Variables
df_fnb = pd.read_csv('fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_halal = pd.read_csv('premis_makanan_list_part_6.csv', encoding='utf-8')


In [22]:
from fuzzywuzzy import process

matches = []  # To store potential matches
THRESHOLD = 90  # Set a threshold for considering a match
MAX_MATCHES = 3 # Maximum number of matches to find

df_fnb = pd.read_csv('fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_halal = pd.read_csv('premis_makanan_list_part_6_cleaned.csv', encoding='utf-8')

# Iterate over each row in df_fnb
for index, fnb_row in tqdm(df_fnb.iterrows(), total=df_fnb.shape[0], desc="Searching for matches"):
    fnb_name = str(fnb_row['Name']).upper()
    fnb_address = str(fnb_row['Address']).upper()

    # Iterate over each row in df_halal to find a match
    for _, halal_row in df_halal.iterrows():
        halal_name = halal_row['Premise Name']
        halal_address = halal_row['Address']

        # Calculate the similarity scores
        name_score = fuzz.token_set_ratio(fnb_name, halal_name)
        address_score = fuzz.token_set_ratio(fnb_address, halal_address)

        # Check if both scores are above the threshold
        if name_score >= THRESHOLD and address_score >= THRESHOLD:
            matches.append((fnb_name, fnb_address, halal_name, name_score, halal_address, address_score))

            # Break if maximum matches are found
            if len(matches) >= MAX_MATCHES:
                break

# Print the matches
for match in matches:
    print(f"FNB Name: {match[0]}, FNB Address: {match[1]}")
    print(f"Matched Halal Name: {match[2]} with score {match[3]}")
    print(f"Matched Halal Address: {match[4]} with score {match[5]}\n")

# for index, fnb_row in tqdm(df_fnb.iterrows(), total=df_fnb.shape[0], desc="Searching for matches"):
#     fnb_name = str(fnb_row['Name']).upper()
#     fnb_address = str(fnb_row['Address']).upper()
#     is_name_matched = False
#     is_address_matched = False

#     if pd.isnull(fnb_address) or fnb_address.strip() == "":
#         continue

#     best_match_name = process.extractOne(fnb_name, df_halal['Premise Name'].tolist(), scorer=fuzz.token_set_ratio)
#     if best_match_name and best_match_name[1] >= THRESHOLD:
#         is_name_matched = True

#     best_match_address = process.extractOne(fnb_address, df_halal['Address'].tolist(), scorer=fuzz.token_set_ratio)
#     if best_match_address and best_match_address[1] >= THRESHOLD:
#         is_address_matched = True
        
#     if is_name_matched == True and is_address_matched == True:
#         matches.append((fnb_name, fnb_address, best_match_name[0], best_match_name[1], best_match_address[0], best_match_address[1]))
#         if len(matches) >= MAX_MATCHES:
#             break

# for match in matches[:10]:
#     print(f"FNB Name: {match[0]}")
#     print(f"Best Premis Name Match: {match[2]} with score {match[3]}")
#     print(f"FNB Address: {match[1]}")
#     print(f"Best Premis Address Match: {match[4]} with score {match[5]}\n")

Searching for matches: 100%|██████████| 4070/4070 [36:15<00:00,  1.87it/s]

FNB Name: 4FINGERS CRISPY CHICKEN @ PUCHONG UTAMA, FNB Address: PUSAT NO, 20 (GF, JALAN PU 7/2, BANDAR PUCHONG UTAMA, 47140 PUCHONG, SELANGOR
Matched Halal Name: 4FINGERS CRISPY CHICKEN PUCHONG UTAMA with score 100
Matched Halal Address: 20 (GF), JALAN PU 7/2,  TAMAN PUCHONG UTAMA, PUCHONG with score 91

FNB Name: 7CAFÉ BY 7-ELEVEN BANDAR PUTERI PUCHONG (#2880), FNB Address: 2, JALAN PUTERI 1/1, BANDAR PUTERI, 47100 PUCHONG, SELANGOR
Matched Halal Name: BANDAR PUTERI with score 100
Matched Halal Address: NO. 15, JALAN PUTERI 2/1 BANDAR PUTERI, PUCHONG SELANGOR DARUL EHSAN with score 93

FNB Name: A&W BMC MALL, FNB Address: LOT G-16, GROUND FLOOR BMC MALL, JALAN TEMENGGUNG 21/9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR
Matched Halal Name: A&W BMC MALL with score 100
Matched Halal Address: LOT G-16, GROUND FLOOR BMC MALL, JALAN TEMENGGUNG 21/9, BANDAR MAHKOTA CHERAS with score 100

FNB Name: A&W C180 CHERAS SELATAN, FNB Address: 3A & 5,JALAN C180/1, BT 11 CHERAS, 43200 BALAKONG, SEL

## Fuzzy Matching - Fuzzy Wuzzy Is Not Working Here ##

In [56]:
# def fuzzy_match(address_to_match, address_list, threshold=90):
#     # Find the best match above the threshold
#     best_match = None
#     best_score = 0
#     for potential_match in address_list:
#         score = fuzz.WRatio(address_to_match, potential_match)
#         if score > threshold and score > best_score:
#             best_match = potential_match
#             best_score = score
#     return best_match

In [57]:
# def normalize_address(address):
#     if pd.isnull(address) or not isinstance(address, str):
#         return address  # Return as is if it's NaN or not a string
#     address = str(address).upper().strip()
#     address = re.sub(r'\s+', ' ', address)  # Replace multiple spaces with a single space
#     address = re.sub(r'[^\w\s]', '', address)  # Remove punctuation
#     return address

# # Apply normalization to both dataframes
# df_fnb['Normalized Address'] = df_fnb['Address'].apply(normalize_address)
# df_halal['Normalized Address'] = df_halal['Address'].apply(normalize_address)

In [69]:
# # Apply fuzzy_match to find best matches above the threshold
# df_fnb['Matched Address'] = df_fnb['Normalized Address'].progress_apply(
#     lambda x: fuzzy_match(x, df_halal['Normalized Address'].tolist(), threshold=80)
# )


100%|██████████| 3816/3816 [37:00<00:00,  1.72it/s] 


In [70]:
# # display (df_fnb)

# df_not_null_address = df_fnb[df_fnb['Matched Address'].notnull()]

# # Display the top 10 rows of the filtered DataFrame
# display (df_not_null_address)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Normalized Address,Matched Address,Matched Address Original
0,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,NaN
1,Malaysian Kitchen,Malaysian Kitchen,Halal Food,Subang Jaya,NaN,"14, Jalan SS 12/1b, Ss 12, 47500 Subang Jaya, ...",03-5887 2805,https://www.google.com/maps/place/Malaysian+Ki...,NaN,https://malaysiankitchen.grab.shop/,https://www.foodpanda.my/restaurant/y2xq/malay...,14 JALAN SS 121B SS 12 47500 SUBANG JAYA SELANGOR,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,NaN
2,E-ONE Chinese Muslim Food Court,E-ONE Chinese Muslim Food Court,Halal Food,Subang Jaya,NaN,"12, Jalan SS 15/8a, Ss 15, 47500 Subang Jaya, ...",016-614 0060,https://www.google.com/maps/place/E-ONE+Chines...,NaN,NaN,NaN,12 JALAN SS 158A SS 15 47500 SUBANG JAYA SELANGOR,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,NaN
3,Suhaimi Chinese Food (Halal),Suhaimi Chinese Food,Halal Food,Subang Jaya,Abang SS19 Food Court,"Ss 19, 40150 Subang Jaya, Selangor",NaN,https://www.google.com/maps/place/Suhaimi+Chin...,NaN,NaN,https://www.foodpanda.my/restaurant/zzu1/suhai...,SS 19 40150 SUBANG JAYA SELANGOR,NO170 PERSIARAN RAJA MUDA MUSA PORT KLANG SELA...,NaN
4,Mohd Chan Dim Sum @ Sunway Geo Avenue,Mohd Chan Dim Sum,Halal Food,Subang Jaya,Sunway GEO Avenue,"A-01-01, Sunway Geo Avenue, Jalan Lagoon Selat...",03-5612 9939,https://www.google.com/maps/place/Mohd+Chan+Di...,NaN,http://www.mohdchan.com.my/,https://www.foodpanda.my/restaurant/xnl2/mohd-...,A0101 SUNWAY GEO AVENUE JALAN LAGOON SELATAN B...,G32 GROUNF FLOOR PERSIARAN BUKIT RAJA BANDAR B...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,Curry Chicken Bun Subang Jaya,Curry Chicken Bun Subang Jaya,Dessert,Subang Jaya,NaN,"Pesiaran, Persiaran Subang Permai, USJ1, 47500...",014-722 1597,https://www.google.com/maps/place/Curry+Chicke...,NaN,http://www.twinklejelly.com/,NaN,PESIARAN PERSIARAN SUBANG PERMAI USJ1 47500 SU...,LOT G30 GROUND FLOOR AEON BUKIT TINGGI SHOPPIN...,NaN
3812,BahWang Traditional Dessert 霸王传统糖水,BahWang Traditional Dessert 霸王传统糖水,Dessert,Subang Jaya,NaN,"No. 31, Tingkat, 1, Jalan USJ 10/1e, Taipan Bu...",NaN,https://www.google.com/maps/place/BahWang+Trad...,NaN,NaN,https://www.foodpanda.my/restaurant/vyk6/bahwa...,NO 31 TINGKAT 1 JALAN USJ 101E TAIPAN BUSINESS...,NO 24 JALAN TAIPING41400 KLANGSELANGOR,NaN
3813,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,Dessert,Subang Jaya,NaN,"Jalan Pinggiran USJ 2/1, Taman Pinggiran Usj, ...",017-905 9614,https://www.google.com/maps/place/SKIN+DESSERT...,NaN,https://skin-dessert-shah-alam.business.site/?...,NaN,JALAN PINGGIRAN USJ 21 TAMAN PINGGIRAN USJ 476...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,NaN
3814,Leong Sweet Dessert Kajang,Leong Sweet Dessert Kajang,Dessert,Subang Jaya,NaN,"Jalan PJS 11/9, Bandar Sunway, 47500 Subang Ja...",011-3173 0013,https://www.google.com/maps/place/Leong+Sweet+...,NaN,NaN,NaN,JALAN PJS 119 BANDAR SUNWAY 47500 SUBANG JAYA ...,LOT PT 3358 SEKSYEN 31PERSIARAN RAJA MUDA MUSA...,NaN


In [71]:
#df_backup = df_fnb

In [73]:
# display( df_backup)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Normalized Address,Matched Address,Matched Address Original
0,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN..."
1,Malaysian Kitchen,Malaysian Kitchen,Halal Food,Subang Jaya,NaN,"14, Jalan SS 12/1b, Ss 12, 47500 Subang Jaya, ...",03-5887 2805,https://www.google.com/maps/place/Malaysian+Ki...,NaN,https://malaysiankitchen.grab.shop/,https://www.foodpanda.my/restaurant/y2xq/malay...,14 JALAN SS 121B SS 12 47500 SUBANG JAYA SELANGOR,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN..."
2,E-ONE Chinese Muslim Food Court,E-ONE Chinese Muslim Food Court,Halal Food,Subang Jaya,NaN,"12, Jalan SS 15/8a, Ss 15, 47500 Subang Jaya, ...",016-614 0060,https://www.google.com/maps/place/E-ONE+Chines...,NaN,NaN,NaN,12 JALAN SS 158A SS 15 47500 SUBANG JAYA SELANGOR,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN..."
3,Suhaimi Chinese Food (Halal),Suhaimi Chinese Food,Halal Food,Subang Jaya,Abang SS19 Food Court,"Ss 19, 40150 Subang Jaya, Selangor",NaN,https://www.google.com/maps/place/Suhaimi+Chin...,NaN,NaN,https://www.foodpanda.my/restaurant/zzu1/suhai...,SS 19 40150 SUBANG JAYA SELANGOR,NO170 PERSIARAN RAJA MUDA MUSA PORT KLANG SELA...,"NO.170, PERSIARAN RAJA MUDA MUSA, PORT KLANG, ..."
4,Mohd Chan Dim Sum @ Sunway Geo Avenue,Mohd Chan Dim Sum,Halal Food,Subang Jaya,Sunway GEO Avenue,"A-01-01, Sunway Geo Avenue, Jalan Lagoon Selat...",03-5612 9939,https://www.google.com/maps/place/Mohd+Chan+Di...,NaN,http://www.mohdchan.com.my/,https://www.foodpanda.my/restaurant/xnl2/mohd-...,A0101 SUNWAY GEO AVENUE JALAN LAGOON SELATAN B...,G32 GROUNF FLOOR PERSIARAN BUKIT RAJA BANDAR B...,"G-32 GROUNF FLOOR, PERSIARAN BUKIT RAJA BANDAR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,Curry Chicken Bun Subang Jaya,Curry Chicken Bun Subang Jaya,Dessert,Subang Jaya,NaN,"Pesiaran, Persiaran Subang Permai, USJ1, 47500...",014-722 1597,https://www.google.com/maps/place/Curry+Chicke...,NaN,http://www.twinklejelly.com/,NaN,PESIARAN PERSIARAN SUBANG PERMAI USJ1 47500 SU...,LOT G30 GROUND FLOOR AEON BUKIT TINGGI SHOPPIN...,"LOT G30, GROUND FLOOR, AEON BUKIT TINGGI SHOPP..."
3812,BahWang Traditional Dessert 霸王传统糖水,BahWang Traditional Dessert 霸王传统糖水,Dessert,Subang Jaya,NaN,"No. 31, Tingkat, 1, Jalan USJ 10/1e, Taipan Bu...",NaN,https://www.google.com/maps/place/BahWang+Trad...,NaN,NaN,https://www.foodpanda.my/restaurant/vyk6/bahwa...,NO 31 TINGKAT 1 JALAN USJ 101E TAIPAN BUSINESS...,NO 24 JALAN TAIPING41400 KLANGSELANGOR,"NO. 24, JALAN TAIPING41400 KLANGSELANGOR"
3813,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,Dessert,Subang Jaya,NaN,"Jalan Pinggiran USJ 2/1, Taman Pinggiran Usj, ...",017-905 9614,https://www.google.com/maps/place/SKIN+DESSERT...,NaN,https://skin-dessert-shah-alam.business.site/?...,NaN,JALAN PINGGIRAN USJ 21 TAMAN PINGGIRAN USJ 476...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN..."
3814,Leong Sweet Dessert Kajang,Leong Sweet Dessert Kajang,Dessert,Subang Jaya,NaN,"Jalan PJS 11/9, Bandar Sunway, 47500 Subang Ja...",011-3173 0013,https://www.google.com/maps/place/Leong+Sweet+...,NaN,NaN,NaN,JALAN PJS 119 BANDAR SUNWAY 47500 SUBANG JAYA ...,LOT PT 3358 SEKSYEN 31PERSIARAN RAJA MUDA MUSA...,"LOT PT 3358, SEKSYEN 31,PERSIARAN RAJA MUDA MU..."


In [72]:
# # Map the original address from df_halal to the matched addresses in df_fnb
# df_fnb['Matched Address Original'] = df_fnb['Matched Address'].map(
#     df_halal.set_index('Normalized Address')['Address'].to_dict()
# )

# # Merge the original DF2 columns based on the matched original address
# merged_df = pd.merge(df_fnb, df_halal, left_on='Matched Address Original', right_on='Address', how='left')

# # Select and rename columns as necessary to fit the final format
# # final_columns = ['FNB Name', 'FNB Address', 'FNB Opening Hours', 'FNB Contact', 'Expiry Date', 'Certification Body']
# # merged_df = merged_df[final_columns]
# display (merged_df)

,Name,Simplified Name,Category,Area,Located In,Address_x,Phone,Google Map Url,Menu Url,Website Url,Order Url,Normalized Address_x,Matched Address,Matched Address Original,Premise Name,Company Brand Name,Address_y,Halal Certification Expiry Date,Halal Certification Body,Normalized Address_y
0,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2017-12-31,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...
1,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2018-02-28,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...
2,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2018-05-15,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...
3,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2018-04-30,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...
4,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...,55A JALAN USJ 101F TAIPAN BUSINESS CENTRE 4762...,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2018-06-30,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43814,Bake With Yen @Subang Jaya,Bake With Yen,Dessert,Subang Jaya,NaN,"B9, Jalan SS 15/8a, Ss 15, 47500 Subang Jaya, ...",03-5612 7747,https://www.google.com/maps/place/Bake+With+Ye...,NaN,http://www.bakewithyen.my/,NaN,B9 JALAN SS 158A SS 15 47500 SUBANG JAYA SELANGOR,LOT NO 9 GROUND FLOORCARREFOUR KLANG SHOPPING ...,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",KFC CARREFOUR KLANG,KFC (PENINSULAR MALAYSIA) SDN BHD,"LOT NO. 9, GROUND FLOORCARREFOUR KLANG SHOPPIN...",2018-09-30,JABATAN KEMAJUAN ISL

In [61]:
# merged_df.describe()

,Name,Simplified Name,Category,Area,Located In,Address_x,Phone,Google Map Url,Menu Url,Website Url,Order Url,Normalized Address_x,Matched Address,Matched Address Original,Premise Name,Company Brand Name,Address_y,Halal Certification Expiry Date,Halal Certification Body,Normalized Address_y
count,3845,3845,3845,3845,910,3844,3281,3845,579,1957,1325,3844,31,31,31,31,31,31,31,31
unique,3781,3695,12,5,472,3815,3161,3815,492,1629,1264,3813,2,2,2,2,2,14,1,2
top,Tastetify cafe,Tastetify cafe,Chinese Food,Cheras,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,https://kfc.com.my/,https://kfc.com.my/,http://kfc.com.my/,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2021-06-30,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
freq,21,21,526,879,22,21,21,21,20,39,23,21,21,21,21,21,21,10,31,21


In [65]:
# df_not_null_premise = merged_df[merged_df['Premise Name'].notnull()]

# # Display the top 10 rows of the filtered DataFrame
# display(df_not_null_premise.tail(10))

,Name,Simplified Name,Category,Area,Located In,Address_x,Phone,Google Map Url,Menu Url,Website Url,Order Url,Normalized Address_x,Matched Address,Matched Address Original,Premise Name,Company Brand Name,Address_y,Halal Certification Expiry Date,Halal Certification Body,Normalized Address_y
3704,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2020-05-15,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
3705,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2021-05-31,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
3706,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2021-06-30,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
3707,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2021-06-30,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
3708,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",SECRET RECIPE CAKES & CAFE SDN BHD - SR @ SUMMIT,SECRET RECIPE CAKES & CAFE SDN BHD,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...",2022-07-31,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM),LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...
3709,Tastetify cafe,Tastetify cafe,Cafe,Subang Jaya,Floor 0 · Summit USJ,"G10, Ground Floor, Summit Usj, Persiaran Kewaj...",011-5110 6232,https://www.google.com/maps/place/Tastetify+ca...,NaN,http://tastetifycafe.com/,https://www.foodpanda.my/restaurant/oewq/taste...,G10 GROUND FLOOR SUMMIT USJ PERSIARAN KEWAJIPA...,LOT NOKLG02 LOWER GROUND FLOOR THE SUMMIT SUBA...,"LOT NO.KLG02, LOWER GROUND FLOOR, THE SUMMIT S...

In [63]:
# merged_df.to_csv('fnb_list_part_7_merged.csv', index=False, encoding='utf-8')